In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import GRU, Dense
from sklearn.metrics import mean_squared_error

# Load the dataset
file_path = '/content/Final.xlsx'
weather_data = pd.read_excel(file_path)
weather_data['date'] = pd.to_datetime(weather_data['date'])
weather_data.set_index('date', inplace=True)

# Resample the data to ensure daily frequency and interpolate missing values
weather_data_daily = weather_data.resample('D').interpolate()

# Normalize the dataset
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(weather_data_daily)
normalized_weather_data = pd.DataFrame(normalized_data, columns=weather_data_daily.columns, index=weather_data_daily.index)

# Function to create sequences for model training
def create_sequences(input_data, look_back=30):
    X, y = [], []
    for i in range(len(input_data) - look_back):
        sequence = input_data[i:(i + look_back)]
        X.append(sequence)
        y.append(input_data[i + look_back, 0])
    return np.array(X), np.array(y)

# Prepare input and output sequences
look_back = 30
X, y = create_sequences(normalized_weather_data.values, look_back=look_back)

# Splitting data into train and test sets
split_percent = 0.80
split = int(split_percent * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Designing the GRU model
model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(GRU(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, verbose=1)

# Model evaluation
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')




Epoch 1/30
84/84 [==============================] - 8s 41ms/step - loss: 0.0276 - val_loss: 0.0052
Epoch 2/30
84/84 [==============================] - 3s 31ms/step - loss: 0.0050 - val_loss: 0.0042
Epoch 3/30
84/84 [==============================] - 3s 38ms/step - loss: 0.0042 - val_loss: 0.0037
Epoch 4/30
84/84 [==============================] - 3s 37ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 5/30
84/84 [==============================] - 3s 31ms/step - loss: 0.0036 - val_loss: 0.0033
Epoch 6/30
84/84 [==============================] - 3s 32ms/step - loss: 0.0035 - val_loss: 0.0032
Epoch 7/30
84/84 [==============================] - 3s 31ms/step - loss: 0.0034 - val_loss: 0.0031
Epoch 8/30
84/84 [==============================] - 4s 45ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 9/30
84/84 [==============================] - 3s 31ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 10/30
84/84 [==============================] - 3s 31ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 11/

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')

R2 Score: 0.9121856495398645


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, r2_score


# Function to predict next N days
def predict_next_days(model, last_days, days_to_predict):
    prediction_list = last_days[-look_back:]

    for _ in range(days_to_predict):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, x.shape[1]))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, np.array([[out] + [0]*(x.shape[2] - 1)]), axis=0)
    prediction_list = prediction_list[look_back:]

    return prediction_list[:, 0]

# Prepare data for prediction
last_days = normalized_weather_data.values[-look_back:]
days_to_predict = 30

# Predict future weather
future_predictions = predict_next_days(model, last_days, days_to_predict)

# Inverse transform the predictions
future_predictions_inversed = scaler.inverse_transform(np.column_stack([future_predictions] + [np.zeros((len(future_predictions), normalized_weather_data.shape[1] - 1))]))

# Dates for visualization
last_date = normalized_weather_data.index[-1]
dates = pd.date_range(start=last_date, periods=days_to_predict + 1, freq='D')[1:]

# Plotting the results
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=future_predictions_inversed[:, 0], mode='lines+markers', name='Predicted Temperature'))
fig.update_layout(title='Weather Forecast for Next 30 Days', xaxis_title='Date', yaxis_title='Average Temperature (°C)', template='plotly_dark')
fig.show()


1/1 [==============================] - 0s 24ms/step
